In [1]:
import requests
import sqlite3
import time
from datetime import datetime

API_TOKEN = "113482992bb97eb41f34ca5cb227fe840631df5faa2eb4f11fc5deb6bfa078ef"
BASE_URL = "https://api.oilpriceapi.com/v1/prices/past_year"
CODES = ["WTI_USD", "BRENT_CRUDE_USD"]  # WTI and Brent benchmarks

def fetch_prices(code):
    headers = {"Authorization": f"Token {API_TOKEN}"}
    params = {"by_code": code}
    resp = requests.get(BASE_URL, headers=headers, params=params)
    resp.raise_for_status()
    return resp.json()["data"]["prices"]

def init_db(db_path="oil_prices.db"):
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    c.execute("""
    CREATE TABLE IF NOT EXISTS oil_prices (
        id INTEGER PRIMARY KEY,
        code TEXT,
        price REAL,
        timestamp INTEGER,
        created_at TEXT
    )""")
    conn.commit()
    return conn

def save_prices(conn, code, prices):
    c = conn.cursor()
    for entry in prices:
        c.execute("""
        INSERT OR IGNORE INTO oil_prices (code, price, timestamp, created_at)
        VALUES (?, ?, ?, ?)
        """, (
            code,
            entry["price"],
            int(datetime.fromisoformat(entry["created_at"].rstrip('Z')).timestamp()),
            entry["created_at"]
        ))
    conn.commit()

def main():
    conn = init_db()
    for code in CODES:
        print(f"Fetching {code}…")
        data = fetch_prices(code)
        print(f"  Retrieved {len(data)} records")
        save_prices(conn, code, data)
        time.sleep(1)  # avoid rate limits
    
    # Example query
    cursor = conn.cursor()
    cursor.execute("""
      SELECT code, datetime(timestamp, 'unixepoch') AS day, AVG(price)
      FROM oil_prices
      GROUP BY code, day
      ORDER BY day DESC LIMIT 5
    """)
    for row in cursor.fetchall():
        print(row)
    conn.close()

if __name__ == "__main__":
    main()


Fetching WTI_USD…
  Retrieved 100 records
Fetching BRENT_CRUDE_USD…
  Retrieved 100 records
('BRENT_CRUDE_USD', '2024-06-19 05:34:13', 85.41)
('WTI_USD', '2024-06-19 05:34:13', 81.43)
('WTI_USD', '2024-06-19 05:33:42', 80.87)
('WTI_USD', '2024-06-19 05:33:06', 81.76)
('BRENT_CRUDE_USD', '2024-06-19 05:33:05', 85.42)


In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("oil_prices.db")

# Load all data into a DataFrame
df = pd.read_sql_query("SELECT * FROM oil_prices", conn)

print(df.head())
conn.close()


   id     code  price   timestamp                created_at
0   1  WTI_USD  81.37  1718771962  2024-06-18T23:39:22.722Z
1   2  WTI_USD  81.26  1718772012  2024-06-18T23:40:12.733Z
2   3  WTI_USD  80.78  1718772041  2024-06-18T23:40:41.528Z
3   4  WTI_USD  80.78  1718772087  2024-06-18T23:41:27.705Z
4   5  WTI_USD  80.78  1718772123  2024-06-18T23:42:03.334Z
